# Анализ экспериментальных данных с работающего ПИД-регулятора

In [ ]:
from nn_laser_stabilizer.paths import get_data_dir

BASE_DATA_DIR_PATH = get_data_dir()
DATA_PATH = BASE_DATA_DIR_PATH / "2026-01-26_2_cut.csv"

print(DATA_PATH)

In [ ]:
import pandas as pd


data = pd.read_csv(DATA_PATH)
display(data)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Загрузка данных (если еще не загружены)
# data = pd.read_csv('your_data.csv')

# Вычисление ошибки и её компонентов
data['error'] = data['process_variable'] - data['setpoint'] 
data['error_integral'] = data['error'].cumsum() 
data['error_derivative'] = data['error'].diff().fillna(0)

# Восстановление коэффициентов
X = data[['error', 'error_integral', 'error_derivative']].values
y = data['control_output'].values

model = LinearRegression(fit_intercept=False)
model.fit(X, y)

Kp, Ki, Kd = model.coef_
print(f"Найденные коэффициенты: Kp={Kp:.4f}, Ki={Ki:.4f}, Kd={Kd:.4f}")

# Применение PID к данным
data['control_output_predicted'] = (
    Kp * data['error'] + 
    Ki * data['error_integral'] + 
    Kd * data['error_derivative']
)

# Сравнение результатов
mse = mean_squared_error(data['control_output'], data['control_output_predicted'])
r2 = r2_score(data['control_output'], data['control_output_predicted'])

print(f"\nМетрики:")
print(f"MSE (Mean Squared Error): {mse:.4f}")
print(f"R² (Coefficient of Determination): {r2:.4f}")
print(f"RMSE (Root Mean Squared Error): {np.sqrt(mse):.4f}")

# Визуализация сравнения
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(data['control_output'], 'b-', label='Реальный control_output', linewidth=2)
plt.plot(data['control_output_predicted'], 'r--', label='Предсказанный PID', linewidth=2, alpha=0.7)
plt.xlabel('Шаг')
plt.ylabel('Control Output')
plt.title('Сравнение реального и предсказанного control_output')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
residuals = data['control_output'] - data['control_output_predicted']
plt.plot(residuals, 'g-', linewidth=1, alpha=0.7)
plt.axhline(y=0, color='k', linestyle='--', linewidth=1)
plt.xlabel('Шаг')
plt.ylabel('Остатки (реальный - предсказанный)')
plt.title('Остатки модели')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Вывод первых нескольких строк для проверки
print("\nПервые 10 строк сравнения:")
print(data[['time', 'process_variable', 'setpoint', 'control_output', 'control_output_predicted', 'error']].head(10))

In [ ]:
df_cut = data.iloc[5335:].reset_index(drop=True)
df_cut.to_csv(BASE_DATA_DIR_PATH / '2026-01-26_2_cut.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(12, 5))
plt.plot(data['process_variable'], 'b-', alpha=0.7, linewidth=0.8, label='Process Variable')
plt.plot(data['setpoint'], color='r', linestyle='--', linewidth=1.5, label='Setpoint')
plt.title('Process Variable over Steps')
plt.xlabel('Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(data['control_output'], 'g-', alpha=0.7, linewidth=0.8, label='Control Output')
plt.xlabel('Step', fontsize=12)
plt.ylabel('Control Output', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='best')
plt.title('Control Output over Steps')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 5))
data['control_output_diff'] = data['control_output'].diff()
plt.plot(data['control_output_diff'], 'b-', alpha=0.7, linewidth=0.8, label='Control Output Delta')
plt.xlabel('Step', fontsize=12)
plt.ylabel('Control Output Change', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xlim(100_000, 101_000)
plt.ylim(-100, 100)
plt.legend(loc='best')
plt.title('Control Output Increments (Differences)')
plt.tight_layout()

In [ ]:
unique_process_variable_values = data['process_variable'].unique()

print(f"Всего уникальных значений: {len(unique_process_variable_values)}")
print(f"Уникальные значения: {sorted(unique_process_variable_values)}")

In [ ]:
unique_control_output_values = data['control_output'].unique()

print(f"Всего уникальных значений: {len(unique_control_output_values)}")
print(f"Уникальные значения: {sorted(unique_control_output_values)}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(data['process_variable'], data['control_output'], 
            alpha=0.5, s=1, edgecolors='none')

plt.xlabel('Process Variable')
plt.ylabel('Control Output')
plt.title('Process Variable vs Control Output')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
window = 500

plt.figure(figsize=(14, 8))
ma = data['process_variable'].rolling(window=window, center=True).mean()
plt.plot(ma, color='purple', linewidth=2, label=f'MA (window={window})', alpha=0.75)
plt.plot(data['setpoint'], color='g', linestyle='--', linewidth=1.5, label='Setpoint', alpha=0.5)
plt.title(f'Moving Average with Window = {window}')
plt.xlabel('Step')
plt.ylabel('Value')
plt.ylim(680, 720)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()